# Incomin Event Handler
  --------------------------------------------------------------------

Handle incoming events and write them to V3IO Stream


## Create and Test a Local Function 
Import nuclio SDK and magics, <b>do not remove the cell and comment !!!</b>

In [2]:
# nuclio: ignore
import nuclio

#### Functions imports

In [3]:
import os
import hashlib
import json
import v3io.dataplane

<b>Specify function dependencies and configuration<b>

In [19]:
%nuclio cmd -c python -m pip install v3io --upgrade
%nuclio env %v3io

In [5]:
%%nuclio env
CONTAINER = users
OUTPUT_STREAM_PATH = /${V3IO_USERNAME}/examples/rapid-churn/incoming-events-stream
SHARDS_COUNT = 8
SHARD_KEY = user_id

%nuclio: setting 'CONTAINER' environment variable
%nuclio: setting 'OUTPUT_STREAM_PATH' environment variable
%nuclio: setting 'SHARDS_COUNT' environment variable
%nuclio: setting 'SHARD_KEY' environment variable


In [6]:
%%nuclio config
spec.triggers.v3io_stream.kind = "v3ioStream"
spec.triggers.v3io_stream.disabled = false
spec.triggers.v3io_stream.url = "http://v3io-webapi:8081/users/${V3IO_USERNAME}/examples/rapid-churn/generated-stream@incomingeventhandler"
spec.triggers.v3io_stream.maxWorkers = 10
spec.triggers.v3io_stream.password = "${V3IO_ACCESS_KEY}"
spec.triggers.v3io_stream.attributes.pollingIntervalMs = 500
spec.triggers.v3io_stream.attributes.seekTo = "earliest"
spec.triggers.v3io_stream.attributes.readBatchSize = 64

%nuclio: setting spec.triggers.v3io_stream.kind to 'v3ioStream'
%nuclio: setting spec.triggers.v3io_stream.disabled to False
%nuclio: setting spec.triggers.v3io_stream.url to 'http://v3io-webapi:8081/users/admin/examples/rapid-churn/generated-stream@incomingeventhandler'
%nuclio: setting spec.triggers.v3io_stream.maxWorkers to 10
%nuclio: setting spec.triggers.v3io_stream.password to '4253f5c9-020f-4e6c-ae66-c9e9c5ad24ed'
%nuclio: setting spec.triggers.v3io_stream.attributes.pollingIntervalMs to 500
%nuclio: setting spec.triggers.v3io_stream.attributes.seekTo to 'earliest'
%nuclio: setting spec.triggers.v3io_stream.attributes.readBatchSize to 64


### Manage output stream

In [14]:
# nuclio: ignore
V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
CONTAINER = os.getenv('CONTAINER')
OUTPUT_STREAM_PATH = os.getenv('OUTPUT_STREAM_PATH')
SHARDS_COUNT = int(os.getenv('SHARDS_COUNT'))
    
v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=V3IO_ACCESS_KEY)

#### Create output stream

In [31]:
# nuclio: ignore
resp = v3io_client.create_stream(container=CONTAINER,
                           path=OUTPUT_STREAM_PATH,
                           shard_count=SHARDS_COUNT)
resp.status_code

204

#### Delete output stream

In [ ]:
# nuclio: ignore
# resp = v3io_client.delete_stream(container=CONTAINER, path=OUTPUT_STREAM_PATH)
# resp.body

## Function code

In [26]:
def init_context(context):
    V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
    CONTAINER = os.getenv('CONTAINER')
    OUTPUT_STREAM_PATH = os.getenv('OUTPUT_STREAM_PATH')
    SHARD_KEY = os.getenv('SHARD_KEY')
    v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=V3IO_ACCESS_KEY)
    
    setattr(context, 'v3io_client', v3io_client)
    setattr(context, 'shard_key', SHARD_KEY)
    setattr(context, 'container', CONTAINER)
    setattr(context, 'output_stream_path', OUTPUT_STREAM_PATH)


def handler(context, event):
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    context.logger.info_with('Got invoked',
                             trigger_kind=event.trigger.kind,
                             event_body=event_dict)
    
    record = event_to_record(event_dict, context.shard_key)
    
    resp = context.v3io_client.put_records(container=context.container, 
                                   path=context.output_stream_path, 
                                   records=[record], 
                                   raise_for_status=v3io.dataplane.RaiseForStatus.never)
    
    context.logger.info_with('Sent event to stream', 
                             record=record,
                             response_status=resp.status_code, 
                             response_body=resp.body.decode('utf-8'))
    
    return resp

def event_to_record(event_dict, shard_key):
    event_str = json.dumps(event_dict)
    return {'data': event_str, 'partition_key': str(event_dict[shard_key])}

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [27]:
# nuclio: end-code
# marks the end of a code section

## Test locally

In [28]:
event = nuclio.Event(body=b'{"user_id" : 111154 , "event_type": "spin"}')
init_context(context)
out = handler(context, event)


Python> 2020-07-28 12:22:03,086 [info] Got invoked: {'trigger_kind': '', 'event_body': {'user_id': 111154, 'event_type': 'spin'}}
Python> 2020-07-28 12:22:03,087 [info] Sent event to stream: {'record': {'data': '{"user_id": 111154, "event_type": "spin"}', 'partition_key': '111154'}, 'response_status': 200, 'response_body': '{ "FailedRecordCount":0,"Records": [{ "SequenceNumber":5006,"ShardId":4 } ] }'}


In [22]:
out.status_code

200

## Deploy function

In [32]:
%nuclio deploy -p rapid-churn -n incoming-event-handler

[nuclio] 2020-07-28 13:52:27,000 (info) Build complete
[nuclio] 2020-07-28 13:52:33,104 (info) Function deploy complete
[nuclio] 2020-07-28 13:52:33,112 done updating incoming-event-handler, function address: 192.168.224.209:31321
%nuclio: function deployed
